# Install libraries

In [ ]:
%pip install xgboost==1.3.1

In [ ]:
%pip install --upgrade google-cloud-core

In [ ]:
%pip install --upgrade google-api-core

In [ ]:
%pip install --upgrade google-cloud-bigquery

In [ ]:
print("Done!")

# Restart kernel

In dropdown menu in top left, select:

    Kernel --> Restart Kernel and Clear All Output

# Test installed packages

In [1]:
import xgboost

# Get the data into [Google Cloud Storage](https://console.cloud.google.com/storage)

<p> 
    1. Create a bucket in <a href="https://console.cloud.google.com/storage/">cloud storage</a> to store the raw data in. Use no weird characters in the name. Remember the name of the bucket.
    
</p>

<p> 
    2. Create a folder named 'instacart' in the bucket
</p>

<p> 
    3. Set parameters
</p>

In [2]:
BUCKET = 'instacart-sebastian-test' # The name of the bucket created above
REGION = 'europe-west1' # Your region of choice
PROJECT = 'avaus-academy' # Do not change
LAB_ID = BUCKET.replace('-', '_').replace('.', '_')

In [3]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['LAB_ID'] = LAB_ID

<p> 
    4. Copy files to your bucket
</p>

In [4]:
%%bash
gsutil -m cp gs://avaus-academy-bucket/instacart/*.csv gs://$BUCKET/instacart/

Copying gs://avaus-academy-bucket/instacart/order_products__prior.csv [Content-Type=application/vnd.ms-excel]...
Copying gs://avaus-academy-bucket/instacart/departments.csv [Content-Type=application/vnd.ms-excel]...
Copying gs://avaus-academy-bucket/instacart/aisles.csv [Content-Type=application/vnd.ms-excel]...
Copying gs://avaus-academy-bucket/instacart/order_products__train.csv [Content-Type=application/vnd.ms-excel]...
Copying gs://avaus-academy-bucket/instacart/orders.csv [Content-Type=application/vnd.ms-excel]...
Copying gs://avaus-academy-bucket/instacart/products.csv [Content-Type=application/vnd.ms-excel]...
/ [6/6 files][680.3 MiB/680.3 MiB] 100% Done                                    
Operation completed over 6 objects/680.3 MiB.                                    


<p> 
    5. Check files are in the bucket you created (not avaus-academy-bucket)
</p>

![cloud_storage](img/instacart_cloud_storage.PNG)

# Read CSV and put into [bigquery](https://console.cloud.google.com/bigquery?project=avaus-academy&p=avaus-academy&page=project)

1. Create a bigquery client to work with

In [5]:
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT)

2. Create a dataset to place the CSV files in

In [6]:
# Name of dataset
dataset_name = '{PROJECT}.instacart_{NAME}'.format(PROJECT=PROJECT, NAME=LAB_ID)

# Create a reference to dataset
dataset = bigquery.Dataset(dataset_name)

# Create the dataset
dataset = client.create_dataset(dataset)
print("Dataset created!")

Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/avaus-academy/datasets?prettyPrint=false: Already Exists: Dataset avaus-academy:instacart_instacart_sebastian_test

3. Create a config to load the CSV files with

In [ ]:
# Job config
job_config = bigquery.LoadJobConfig()
job_config.autodetect = True # Autoinfers the schema from the CSV

# Files to load
files = [
    'aisles.csv',
    'departments.csv',
    'order_products__prior.csv',
    'order_products__train.csv',
    'orders.csv',
    'products.csv',
]

4. Load all files

In [ ]:
for file in files:
    # Build input path and destination table
    input_path = "gs://{BUCKET}/instacart/{FILE}".format(BUCKET=BUCKET, FILE=file)
    table_name = file.split('.')[0] # Take the name before '.csv' as the name of the table
    table = dataset.table(table_name)

    # Create a job for loading the CSV to bigquery
    load_job = client.load_table_from_uri(
        source_uris=input_path, 
        destination=table, 
        job_config=job_config
    )
    print("Starting job for loading {FILE} with id={JOB_ID}".format(FILE=file, JOB_ID=load_job.job_id))

    # Waits for table load to complete.
    load_job.result()  
    print("Job finished.")

    # Check stats for the table
    destination_table = client.get_table(dataset.table(table_name))
    print("Loaded {} rows.".format(destination_table.num_rows))
    print("")
    
print("Finished loading all tables!")

# 5. Check tables are there in BigQuery

# Explore data

#### 1. Look at the table we are going to use: orders

In [ ]:
client.query(
"""
    SELECT
        *
    FROM instacart_{LAB_ID}.orders
    LIMIT 5
""".format(LAB_ID=LAB_ID)
).to_dataframe()

#### 2. Overall stats for table

In [ ]:
client.query(
"""
    SELECT
        COUNT(*) AS nr_orders,
        COUNT(DISTINCT user_id) AS nr_customers,
        MIN(order_number) AS min_order_nr,
        MAX(order_number)  AS max_order_nr,
        MIN(days_since_prior_order) AS min_days,
        MAX(days_since_prior_order) AS max_days
    FROM instacart_{LAB_ID}.orders
""".format(LAB_ID=LAB_ID)
).to_dataframe()

~3.5 million orders

~200k customers

No customer has made more than 100 purchases

the column *days_since_prior_order* is censored after 30 days

#### 3. What are the different values for *eval_set*?

In [ ]:
client.query(
"""
    SELECT
        eval_set,
        COUNT(*) AS nr_orders
    FROM instacart_{LAB_ID}.orders
    GROUP BY 
        eval_set
""".format(LAB_ID=LAB_ID)
).to_dataframe()

We are going to use *prior* for training features and *train* for training targel label

#### 4. Dig deeper on customer level

In [ ]:
client.query(
"""
    SELECT
        user_id,
        COUNT(*) AS nr_orders,
        MIN(CASE WHEN eval_set = 'prior' THEN order_number ELSE NULL END) AS min_order_nr_prior,
        MAX(CASE WHEN eval_set = 'prior' THEN order_number ELSE NULL END) AS max_order_nr_prior,
        MIN(CASE WHEN eval_set = 'train' THEN order_number ELSE NULL END) AS min_order_nr_train,
        MAX(CASE WHEN eval_set = 'train' THEN order_number ELSE NULL END) AS max_order_nr_train    FROM instacart_{LAB_ID}.orders
    GROUP BY 
        user_id
    HAVING min_order_nr_train IS NOT NULL
    LIMIT 5
""".format(LAB_ID=LAB_ID)
).to_dataframe()

*eval_set = 'prior'* contains all transactions prior to the latest one

*eval_set = 'train'* contains the latest transaction

# Create a dataset for ML

Problem:
Given a user and their latest order, predict how long it will be until the next order

#### 1. Create a table with overall features for each user

In [ ]:
user_features="""
    CREATE TABLE instacart_{LAB_ID}.user_features AS
    SELECT
        user_id,
        COUNT(order_id) AS nr_orders,
        SUM(days_since_prior_order) AS user_lifetime,
        COALESCE(COUNT(order_id) / NULLIF(SUM(days_since_prior_order), 0), 1) AS nr_orders_per_day,
        AVG(days_since_prior_order) AS avg_nr_days_between_orders,
        COUNT(CASE WHEN order_dow = 0 THEN order_id END) AS nr_orders_saturday,
        COUNT(CASE WHEN order_dow = 1 THEN order_id END) AS nr_orders_sunday,
        COUNT(CASE WHEN order_dow = 2 THEN order_id END) AS nr_orders_monday,
        COUNT(CASE WHEN order_dow = 3 THEN order_id END) AS nr_orders_tuesday,
        COUNT(CASE WHEN order_dow = 4 THEN order_id END) AS nr_orders_wednesday,
        COUNT(CASE WHEN order_dow = 5 THEN order_id END) AS nr_orders_thursday,
        COUNT(CASE WHEN order_dow = 6 THEN order_id END) AS nr_orders_friday,
        COUNT(CASE WHEN order_hour_of_day BETWEEN 5 AND 11 THEN order_id END) AS nr_orders_morning,
        COUNT(CASE WHEN order_hour_of_day BETWEEN 12 AND 17 THEN order_id END) AS nr_orders_afternoon,
        COUNT(CASE WHEN order_hour_of_day BETWEEN 18 AND 22 THEN order_id END) AS nr_orders_evening,
        COUNT(CASE WHEN order_hour_of_day > 22 OR order_hour_of_day < 5 THEN order_id END) AS nr_orders_night 
    FROM instacart_{LAB_ID}.orders
    WHERE eval_set = 'prior' -- Only use the prior data for training features
    GROUP BY
        user_id
    """.format(LAB_ID=LAB_ID)

In [ ]:
# Drop if it already exists
query_drop = client.query("DROP TABLE IF EXISTS instacart_{LAB_ID}.user_features".format(LAB_ID=LAB_ID))
query_drop.result()  # Wait for finish
print("Dropped table")

# Create table
query_create = client.query(user_features)
query_create.result()  # Wait for finish
print("Created table")

#### 2. Create a table with features for latest transaction

In [ ]:
latest_transaction="""
    CREATE TABLE instacart_{LAB_ID}.latest_transaction AS
    SELECT
        user_id,
        ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY order_number DESC) AS order_rank,
        days_since_prior_order,
        CASE WHEN order_dow = 0 THEN 1 ELSE 0 END AS is_saturday_order,
        CASE WHEN order_dow = 1 THEN 1 ELSE 0 END AS is_sunday_order,
        CASE WHEN order_dow = 2 THEN 1 ELSE 0 END AS is_monday_order,
        CASE WHEN order_dow = 3 THEN 1 ELSE 0 END AS is_tuesday_order,
        CASE WHEN order_dow = 4 THEN 1 ELSE 0 END AS is_wednesday_order,
        CASE WHEN order_dow = 5 THEN 1 ELSE 0 END AS is_thursday_order,
        CASE WHEN order_dow = 6 THEN 1 ELSE 0 END AS is_friday_order,
        CASE WHEN order_hour_of_day BETWEEN 5 AND 11 THEN 1 ELSE 0 END AS is_morning_order,
        CASE WHEN order_hour_of_day BETWEEN 12 AND 17 THEN 1 ELSE 0 END AS is_afternoon_order,
        CASE WHEN order_hour_of_day BETWEEN 18 AND 22 THEN 1 ELSE 0 END AS is_evening_order,
        CASE WHEN order_hour_of_day > 22 OR order_hour_of_day < 5 THEN 1 ELSE 0 END AS is_night_order
    FROM instacart_{LAB_ID}.orders
    WHERE eval_set = 'prior'
""".format(LAB_ID=LAB_ID)

In [ ]:
# Drop if it already exists
query_drop = client.query("DROP TABLE IF EXISTS instacart_{LAB_ID}.latest_transaction".format(LAB_ID=LAB_ID))
query_drop.result()  # Wait for finish
print("Dropped table")

# Create table
query_create = client.query(latest_transaction)
query_create.result()  # Wait for finish
print("Created table")

#### 3. Create a table with features and target label used for modelling

In [ ]:
feature_set="""
    CREATE TABLE instacart_{LAB_ID}.feature_set AS
    SELECT
        -- Observation key
        lt.user_id,
        
        -- Features about last order
        lt.is_saturday_order,
        lt.is_sunday_order,
        lt.is_monday_order,
        lt.is_tuesday_order,
        lt.is_wednesday_order,
        lt.is_thursday_order,
        lt.is_friday_order,
        lt.is_morning_order,
        lt.is_afternoon_order,
        lt.is_evening_order,
        lt.is_night_order,
        lt.days_since_prior_order,
        
        -- Features about user
        uf.nr_orders,
        uf.user_lifetime,
        uf.nr_orders_per_day,
        uf.avg_nr_days_between_orders,
        uf.nr_orders_saturday,
        uf.nr_orders_sunday,
        uf.nr_orders_monday,
        uf.nr_orders_tuesday,
        uf.nr_orders_wednesday,
        uf.nr_orders_thursday,
        uf.nr_orders_friday,
        uf.nr_orders_morning,
        uf.nr_orders_afternoon,
        uf.nr_orders_evening,
        uf.nr_orders_night,
        
        -- Target label
        target.days_since_prior_order AS days_to_next_order,
        
        -- Train vs test
        RAND() <= 0.8 AS is_train
    FROM instacart_{LAB_ID}.latest_transaction lt
    INNER JOIN instacart_{LAB_ID}.user_features uf ON uf.user_id = lt.user_id
    INNER JOIN instacart_{LAB_ID}.orders target ON target.user_id = lt.user_id 
        AND target.eval_set = 'train' 
    WHERE lt.order_rank = 1 -- Take last transaction in prior set
""".format(LAB_ID=LAB_ID)

In [ ]:
# Drop if it already exists
query_drop = client.query("DROP TABLE IF EXISTS instacart_{LAB_ID}.feature_set".format(LAB_ID=LAB_ID))
query_drop.result()  # Wait for finish
print("Dropped table")

# Create table
query_create = client.query(feature_set)
query_create.result()  # Wait for finish
print("Created table")

#### 4. Check tables are there

![bigquery](img/instacart_bigquery.PNG)

# Train a ML model

1. Read the training data into a dataframe

In [7]:
# Read data
df = client.query(
"""
    SELECT
        *
    FROM instacart_{LAB_ID}.feature_set
    WHERE is_train
""".format(LAB_ID=LAB_ID)
).to_dataframe()

2. Create training dataset for xgboost

In [8]:
import xgboost as xgb

# Some variables
drop_columns = ['user_id']  # Columns not to include in training
target_label = 'days_to_next_order'

# Create dataset
features = df.drop(drop_columns + [target_label], axis=1)
labels = df[target_label]
training_dataset = xgb.DMatrix(features, labels)

3. Train 2 models

In [9]:
# train model
model1 = xgb.train({}, training_dataset, 10)
model2 = xgb.train({}, training_dataset, 25)

4. Evaluate 2 models

In [10]:
# Read evaluation data
df_eval = client.query(
"""
    SELECT
        *
    FROM instacart_{LAB_ID}.feature_set
    WHERE NOT is_train 
""".format(LAB_ID=LAB_ID)
).to_dataframe()

In [11]:
# Create evaluation dataset
features_eval = df_eval.drop(drop_columns + [target_label], axis=1)
labels_eval = df_eval[target_label]
eval_dataset = xgb.DMatrix(features_eval, labels_eval)

In [12]:
# Predict with models on evaluation dataset
prediction_model1 = model1.predict(eval_dataset)
prediction_model2 = model2.predict(eval_dataset)

In [13]:
# Evaluate models
from sklearn.metrics import mean_squared_error

rmse_model1 = mean_squared_error(labels_eval, prediction_model1, squared=False)
rmse_model2 = mean_squared_error(labels_eval, prediction_model2, squared=False)

5. Pick best performing model

In [14]:
# Pick best performing model
print("RMSE Model 1: %0.4f" % rmse_model1)
print("RMSE Model 2: %0.4f" % rmse_model2)
if rmse_model1 <= rmse_model2:
    print("Best model is Model 1")
    best_model = model1
else:
    print("Best model is Model 2")
    best_model = model2

RMSE Model 1: 9.1836
RMSE Model 2: 9.1771
Best model is Model 2


6. Save the model to the disk on this jupyterlab instance

In [15]:
# Save model
from google.cloud import storage
model_name = 'model.bst'
best_model.save_model(model_name)

7. Create a folder in Google Cloud Storage to upload the model to

In [16]:
# Create folder for the model in GCS
storage_client = storage.Client(project=PROJECT)
bucket = storage_client.bucket(bucket_name=BUCKET)

folder = bucket.blob('instacart/model/')
folder.upload_from_string('')

8. Upload the model to [Google Cloud Storage](https://console.cloud.google.com/storage)

In [17]:
# Upload the model to GCS
blob = bucket.blob('instacart/model/{MODEL_NAME}'.format(MODEL_NAME=model_name))
blob.upload_from_filename(model_name)

![model](img/instacart_model.PNG)

# Deploy model

1. Some variables needed

In [18]:
MODEL_PATH = "gs://{BUCKET}/instacart/model".format(BUCKET=BUCKET)
MODEL_NAME = "xgboost_model2_{LAB_ID}".format(LAB_ID=LAB_ID)
VERSION = "v1_0_0"
FRAMEWORK = "xgboost"

In [19]:
os.environ['MODEL_PATH'] = MODEL_PATH
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['VERSION'] = VERSION
os.environ['FRAMEWORK'] = FRAMEWORK

2. Create a model and a version of the model in AI Platform

In [20]:
%%bash

# Create placeholder for model
gcloud ai-platform models create $MODEL_NAME --region=$REGION

Using endpoint [https://europe-west1-ml.googleapis.com/]
Created ai platform model [projects/avaus-academy/models/xgboost_model2_instacart_sebastian_test].


In [21]:
%%bash

# Create a version of the model
gcloud ai-platform versions create $VERSION \
  --model $MODEL_NAME \
  --origin $MODEL_PATH \
  --region $REGION \
  --runtime-version=2.4 \
  --framework $FRAMEWORK \
  --python-version=3.7

Using endpoint [https://europe-west1-ml.googleapis.com/]
Creating version (this might take a few minutes)......
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [22]:
%%bash

# Check model
gcloud ai-platform versions describe $VERSION \
  --model $MODEL_NAME --region $REGION

createTime: '2021-08-09T14:39:23Z'
deploymentUri: gs://instacart-sebastian-test/instacart/model
etag: l8SWSQu2LDM=
framework: XGBOOST
isDefault: true
machineType: n1-standard-2
name: projects/avaus-academy/models/xgboost_model2_instacart_sebastian_test/versions/v1_0_0
pythonVersion: '3.7'
runtimeVersion: '2.4'
state: READY


Using endpoint [https://europe-west1-ml.googleapis.com/]


3. Check models in AI platform

![model](img/instacart_ai_platform.PNG)

# Predict using API

In [23]:
import google.auth
import google.auth.transport.requests
creds, project = google.auth.default()

# creds.valid is False, and creds.token is None
# Need to refresh credentials to populate those

auth_req = google.auth.transport.requests.Request()
creds.refresh(auth_req)
token = creds.token

In [24]:
import requests
import json
# Construct the API url
api = "https://{REGION}-ml.googleapis.com/v1/projects/{PROJECT}/models/{MODEL_NAME}/versions/{VERSION}:predict"\
    .format(REGION=REGION, PROJECT=PROJECT, MODEL_NAME=MODEL_NAME, VERSION=VERSION)
print(api)
# Header and data field of HTTP request
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
      # True value of target label "days_to_next_order": 8.0
      [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 8.0, 20, 126.0, 0.15873015873015872, 6.631578947368422, 2, 1, 1, 2, 6, 3, 5, 6, 14, 0, 0],
      
      # True value of target label "days_to_next_order": 18.0
      [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 30.0, 3, 60.0, 0.05, 30.0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 2, 0]
  ]
}

# Call the API
response = requests.post(api, json=data, headers=headers)
print(response.content)

https://europe-west1-ml.googleapis.com/v1/projects/avaus-academy/models/xgboost_model2_instacart_sebastian_test/versions/v1_0_0:predict
b'{"predictions": [10.727479934692383, 24.83864402770996]}'


# Clean up what we just did

1. Delete the version of the model and the model

In [25]:
%%bash
gcloud ai-platform versions delete $VERSION --model=$MODEL_NAME --region=$REGION
gcloud ai-platform models delete $MODEL_NAME --region=$REGION

Using endpoint [https://europe-west1-ml.googleapis.com/]
This will delete version [v1_0_0]...

Do you want to continue (Y/n)?  Please enter 'y' or 'n':  
Deleting version [v1_0_0]......
...............................................................................done.


2. Delete the dataset

In [26]:
delete = client.delete_dataset(dataset, delete_contents=True)

3. Delete bucket

In [27]:
%%bash
gsutil rm -r gs://$BUCKET

Removing gs://instacart-sebastian-test/instacart/aisles.csv#1628519874227084...
Removing gs://instacart-sebastian-test/instacart/departments.csv#1628519874222873...
Removing gs://instacart-sebastian-test/instacart/model/#1628519918891043...     
Removing gs://instacart-sebastian-test/instacart/model/model.bst#1628519921200362...
/ [4 objects]                                                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rm ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Removing gs://instacart-sebastian-test/instacart/order_products__prior.csv#1628519874255209...
Removing gs://instacart-sebastian-test/instacart/order_products__train.csv#1628519874222768...
Removing gs://instacart-sebastian-test/instacart/orders.csv#1628519874227162... 
Removing gs://instacart-sebastian-test/instacart/products.csv#162

4. Shutdown this jupyterlab instance 